In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft2, ifft2
from scipy.integrate import odeint
from scipy.sparse import spdiags
from scipy.integrate import solve_ivp
import time

In [2]:
tspan = np.arange(0,4.5,0.5)
nu = 0.001
nx, ny = 64, 64
Lx, Ly = 20, 20
N = nx * ny
x2 = np.linspace(-Lx/2, Lx/2, nx+1)
x = x2[:nx]
y2 = np.linspace(-Ly/2, Ly/2, ny+1)
y = y2[:ny]
X, Y = np.meshgrid(x, y)
w0 = 1 * np.exp(-X**2 - Y**2/20) 
w2 = w0.flatten()
kx = (2 * np.pi / Lx) * np.concatenate((np.arange(0, nx/2), np.arange(-nx/2, 0)))
kx[0] = 1e-6
ky = (2 * np.pi / Ly) * np.concatenate((np.arange(0, ny/2), np.arange(-ny/2, 0)))
ky[0] = 1e-6
KX, KY = np.meshgrid(kx, ky)
K = KX**2 + KY**2
m = 64
n = m * m 
dx = 20/64
e0 = np.zeros((n,1)); e1 = np.ones((n,1)); e2 = np.copy(e1); e4 = np.copy(e0) #Copy of vector of ones
for j in range(1, m+1):
    e2[m*j-1] = 0 #every m^th as zero
    e4[m*j-1] = 1 #every m^th value as 1
e3 = np.zeros_like(e2); e3[1:n] = e2[0:n-1]; e3[0] = e2[n-1]
e5 = np.zeros_like(e4); e5[1:n] = e4[0:n-1]; e5[0] = e4[n-1]
diagonals = [e1.flatten(), e1.flatten(), e5.flatten(),
             e2.flatten(), -4*e1.flatten(), e3.flatten(),
             e4.flatten(), e1.flatten(), e1.flatten()]
offsets = [-(n-m), -m, -m+1, -1, 0, 1, m-1, m, (n-m)]
matA = spdiags(diagonals, offsets, n, n).toarray()
A = matA/(dx**2)
A[0,0] = 2 / (dx**2)
m = 64
n = m * m
e0 = np.zeros((n,1))
e1 = np.ones((n,1))
e2 = np.copy(e1)
e2[1:n] = e1[0:n-1]
e2[0] = e2[n-1]
diagonals = [e2.flatten(), -e1.flatten(), e0.flatten(), e1.flatten(), -e2.flatten()]
offsets = [-(n-m), -m, 0, m, (n-m)]
np.shape(diagonals)
matB = spdiags(diagonals, offsets ,n,n).toarray()
B = matB/(dx*2)
m =64
n =  m * m
e0 = np.zeros((n,1)); e1 = np.ones((n,1)); e2 = np.copy(e1)
for j in range(1, m+1):
    e0[m*j-1] = 1 #every m^th as zero
    e1[m*j-1] = 0
e3 = np.zeros_like(e0); e3[1:n] = e0[0:n-1]; e3[0] = e0[n-1]
e4 = np.zeros_like(e1); e4[1:n] = e1[0:n-1]; e4[0] = e4[n-1]
diagonals = [e3.flatten(), -e1.flatten(), e4.flatten(), -e0.flatten()]
offsets = [-(m-1), -1, 1, (m-1)]
np.shape(diagonals)
matC = spdiags(diagonals, offsets ,n,n).toarray() / (2*dx)
C = matC

In [14]:
# Define the ODE system
start_time = time.time()
def spc_rhs(t,w2):
    w = w2.reshape((nx,ny))
    wt = fft2(w)
    psit = -wt / K
    psi = np.real(ifft2(psit)).flatten()
    rhs = nu*np.dot(w2,A) + np.dot(C,psi)*np.dot(B,w2) - np.dot(B,psi)*np.dot(C,w2)
    return rhs

w_sol = solve_ivp(spc_rhs,[tspan[0], tspan[-1]], w2, t_eval=tspan, method='RK45')
A1 = w_sol.y
end_time = time.time()  # Record the end time
elapsed_time = end_time - start_time
print(f"FFT Elapsed time: {elapsed_time:.2f} seconds")

A1

#fig, ax=plt.subplots()
#fftplot = plt.color()
#plt.pcolor(x, y, w2, shading='interp')

FFT Elapsed time: 0.61 seconds


array([[2.50656748e-46, 3.59416114e-45, 1.89181856e-44, ...,
        1.90731975e-42, 4.82357855e-42, 1.14878189e-41],
       [1.17762859e-43, 6.53319073e-43, 2.62245765e-42, ...,
        1.58919524e-40, 3.56490552e-40, 7.46983929e-40],
       [4.55107657e-41, 1.92934361e-40, 6.43507373e-40, ...,
        2.49615609e-38, 5.08470403e-38, 9.69547626e-38],
       ...,
       [1.96785570e-38, 1.23357409e-37, 5.47591944e-37, ...,
        4.35301636e-35, 1.02350494e-34, 2.23002255e-34],
       [6.19028421e-41, 5.33777602e-40, 2.93883953e-39, ...,
        3.67929755e-37, 9.45766657e-37, 2.23852330e-36],
       [1.60178709e-43, 1.99380943e-42, 1.41671738e-41, ...,
        2.79951703e-39, 7.95938807e-39, 2.07769586e-38]])

In [4]:
start_time = time.time()
def back_rhs(t,w2):
    psi = np.linalg.solve(A,w2).flatten()
    rhs = nu*np.dot(w2,A) + np.dot(C,psi)*np.dot(B,w2) - np.dot(B,psi)*np.dot(C,w2)
    return rhs.flatten()
#w = w2.reshape((nx,ny))
#np.linalg.solve(A,w2)
B_sol = solve_ivp(back_rhs, [tspan[0], tspan[-1]], w2, t_eval=tspan, method='RK45')
A2 = np.real(B_sol.y)
end_time = time.time()  # Record the end time
elapsed_time = end_time - start_time
print(f"A Backslash B Elapsed time: {elapsed_time:.2f} seconds")
    

A Backslash B Elapsed time: 13.28 seconds


In [5]:
from scipy.linalg import lu, solve_triangular
start_time = time.time()
def lu_rhs(t,w2):
    P, L, U = lu(A)
    Pb = np.dot(P, w2)
    y = solve_triangular(L, Pb, lower=True)
    psi = solve_triangular(U, y)
    rhs = nu*np.dot(w2,A) + np.dot(C,psi)*np.dot(B,w2) - np.dot(B,psi)*np.dot(C,w2)
    return rhs
C_sol = solve_ivp(lu_rhs, [tspan[0], tspan[-1]], w2, t_eval=tspan, method='RK45')
A3 = np.real(C_sol.y)
end_time = time.time()  # Record the end time
elapsed_time = end_time - start_time
print(f"LU Elapsed time: {elapsed_time:.2f} seconds")
    

LU Elapsed time: 21.32 seconds


In [8]:
A3

array([[ 2.50656748e-46, -1.78630705e-36,  1.02614869e-36, ...,
        -5.75086257e-26, -6.68013806e-25,  3.02201851e-24],
       [ 1.17762859e-43, -2.26093387e-29,  1.85614646e-29, ...,
        -1.48306461e-19, -7.82424220e-19,  1.67449651e-18],
       [ 4.55107657e-41, -4.24661881e-29,  3.48720993e-29, ...,
        -2.20165937e-20, -2.91637720e-19,  1.23961335e-18],
       ...,
       [ 1.96785570e-38,  7.20814729e-28, -6.58759786e-28, ...,
         4.01674620e-20,  1.34371413e-19, -8.94163775e-19],
       [ 6.19028421e-41,  1.69117621e-28, -1.49205998e-28, ...,
         3.00460834e-20,  1.76009486e-19, -9.80736832e-19],
       [ 1.60178709e-43,  5.05834665e-29, -4.38046443e-29, ...,
         2.47633369e-20,  1.15181522e-20,  3.12761254e-20]])

In [11]:
from scipy.sparse.linalg import bicgstab, gmres
start_time = time.time()
def bstab(t,w2):
    psi,info = bicgstab(A, w2, tol=1e-6)
    rhs = nu*np.dot(w2,A) + np.dot(C,psi)*np.dot(B,w2) - np.dot(B, psi)*np.dot(C,w2)
    return rhs.flatten()
D_sol = solve_ivp(bstab, [tspan[0], tspan[-1]], w2, t_eval=tspan, method='RK45')
bstab_sol = D_sol.y
end_time = time.time()  # Record the end time
elapsed_time = end_time - start_time
print(f"Bicgstab Elapsed time: {elapsed_time:.2f} seconds")

/var/folders/kw/6smh7_210hl2r74h8n87zx_m0000gn/T/ipykernel_54557/3146191653.py:4: DeprecationWarning: 'scipy.sparse.linalg.bicgstab' keyword argument `tol` is deprecated in favor of `rtol` and will be removed in SciPy v1.14.0. Until then, if set, it will override `rtol`.
  psi,info = bicgstab(A, w2, tol=1e-6)


Bicgstab Elapsed time: 56.64 seconds


In [12]:
bstab_sol

array([[ 2.50656748e-46, -1.78634735e-36,  1.02617482e-36, ...,
        -5.74895143e-26, -6.68020232e-25,  3.02199464e-24],
       [ 1.17762859e-43, -2.26092808e-29,  1.85614346e-29, ...,
        -1.48258274e-19, -7.82453346e-19,  1.67446810e-18],
       [ 4.55107657e-41, -4.24660036e-29,  3.48719840e-29, ...,
        -2.20093310e-20, -2.91637366e-19,  1.23958645e-18],
       ...,
       [ 1.96785570e-38,  7.20812190e-28, -6.58757845e-28, ...,
         4.01544265e-20,  1.34382881e-19, -8.94154743e-19],
       [ 6.19028421e-41,  1.69116929e-28, -1.49205504e-28, ...,
         3.00362800e-20,  1.76016036e-19, -9.80721870e-19],
       [ 1.60178709e-43,  5.05832695e-29, -4.38045151e-29, ...,
         2.47553537e-20,  1.15274640e-20,  3.12711812e-20]])

In [9]:
start_time = time.time()
def gres(t,w2):
    psi,info = gmres(A, w2, tol=1e-6)
    rhs = nu*np.dot(w2,A) + np.dot(C,psi)*np.dot(B,w2) - np.dot(B, psi)*np.dot(C,w2)
    return rhs
D_sol = solve_ivp(gres, [tspan[0], tspan[-1]], w2, t_eval=tspan, method='RK45')
end_time = time.time()  # Record the end time
elapsed_time = end_time - start_time
print(f"Gmres Elapsed time: {elapsed_time:.2f} seconds")

/var/folders/kw/6smh7_210hl2r74h8n87zx_m0000gn/T/ipykernel_54557/436517904.py:3: DeprecationWarning: 'scipy.sparse.linalg.gmres' keyword argument `tol` is deprecated in favor of `rtol` and will be removed in SciPy v1.14.0. Until then, if set, it will override `rtol`.
  psi,info = gmres(A, w2, tol=1e-6)


Gmres Elapsed time: 561.70 seconds


In [10]:
gres_sol = D_sol.y
gres_sol

array([[ 2.50656748e-46, -1.78625063e-36,  1.02611761e-36, ...,
        -5.75011599e-26, -6.67970609e-25,  3.02183221e-24],
       [ 1.17762859e-43, -2.26083397e-29,  1.85606084e-29, ...,
        -1.48288603e-19, -7.82376205e-19,  1.67439869e-18],
       [ 4.55107657e-41, -4.24645332e-29,  3.48706727e-29, ...,
        -2.20136961e-20, -2.91617811e-19,  1.23952604e-18],
       ...,
       [ 1.96785570e-38,  7.20794430e-28, -6.58740196e-28, ...,
         4.01628800e-20,  1.34366020e-19, -8.94117600e-19],
       [ 6.19028421e-41,  1.69112348e-28, -1.49201092e-28, ...,
         3.00425820e-20,  1.76000182e-19, -9.80678385e-19],
       [ 1.60178709e-43,  5.05816792e-29, -4.38030123e-29, ...,
         2.47611218e-20,  1.15232327e-20,  3.12608904e-20]])